In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from binning import bin

Read the train and test datasets


In [4]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Get the training features and label


In [5]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [6]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [7]:
kf = KFold(n_splits=5)

## MLP Classifier


Bin the label


In [8]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Convert the label to categorical


In [9]:
y_train_binned_categorical = to_categorical(y_train_binned)

Iniatialize an MLP Classifier model


In [10]:
mlp_class = Sequential()
mlp_class.add(Dense(256, activation="relu"))
mlp_class.add(Dense(len(np.unique(y_train_binned)), activation="softmax"))
mlp_class.compile(optimizer="adam", loss="categorical_crossentropy")

Define an EarlyStopping object


In [11]:
early_stopping = EarlyStopping(patience=20)

Initialize scores lists


In [12]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [13]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    mlp_class.fit(
        X_train[train],
        y_train_binned_categorical[train],
        epochs=100,
        batch_size=32,
        validation_data=(X_train[test], y_train_binned_categorical[test]),
        callbacks=[early_stopping],
        verbose=0,
    )

    # Predict on the test set
    predictions = np.argmax(mlp_class.predict(X_train[test]), axis=1)

    # Evaluate the model
    precision = precision_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train_binned[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
Precision = 0.6189721392131949
Recall = 0.6373599294459431
Accuracy = 0.6373599294459431
F1 score = 0.531217903702812

##### FOLD: 2 #####
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
Precision = 0.6137676123116464
Recall = 0.6366937732643347
Accuracy = 0.6366937732643347
F1 score = 0.5270344647778376

##### FOLD: 3 #####
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
Precision = 0.6166384764140678
Recall = 0.642763575995746
Accuracy = 0.642763575995746
F1 score = 0.5925635391886834

##### FOLD: 4 #####
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
Precision = 0.6218894853404998
Recall = 0.646537748206944
Accuracy = 0.646537748206944
F1 score = 0.5821798505191533

##### FOLD: 5 #####
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Precision = 0.6183323263971833
Recall = 0.642841393979482
Accuracy = 0.642841393979482
F1 score = 0.571950053796778



Print the mean scores of the folds


In [14]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.6179200079353184
Mean Recall = 0.64123928417849
Mean Accuracy = 0.64123928417849
Mean F1 score = 0.5609891623970528


Make predictions on the test set


In [15]:
predictions = np.argmax(mlp_class.predict(X_test), axis=1)

3012/3012 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


Evaluate the model


In [16]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.54      0.15      0.23     35520
           1       0.65      0.93      0.76     60859

    accuracy                           0.64     96379
   macro avg       0.59      0.54      0.50     96379
weighted avg       0.61      0.64      0.57     96379

